# Notebook 2: Data Engineering

<font color='red'>**Wichtig**:  
Dieses Notebook muss spätestens am Donnerstag den **11.11.2021** über die zugehörige Aufgabe auf Moodle abgegeben werden.</font>\
**Kompilieren** Sie vor der Abgabe das Notebook noch einmal **komplett** und speichern Sie es dann ab (_Kernel_ --> _Restart Kernel and Run All Cells_).\
Bevor Sie Ihr bearbeitetes Notebook hochladen, **benennen** Sie das Dokument bitte wie folgt: **_Nachname_Matrikelnr_Notebooknr.ipynb_**

Führen Sie nun nach und nach den unten stehenden Code aus.

## Import gebräuchlicher Bibliotheken

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Zeigt Plots direkt im Notebook an:
%matplotlib inline
import pandas as pd
import sklearn as sk

Insbesondere Pandas DataFrames sind sehr praktisch für den Umgang mit tabellularen Daten.
Eine kleine Einführung in Pandas ist z.B. hier zu finden:\
https://www.python-kurs.eu/pandas.php

## Anwendung: California Housing Dataset

Als Beispieldaten werden wir auf einer Kopie des **California Housing Dataset** arbeiten, welcher 1990 vom US Census erhoben wurde. Der Datensatz ist bei der StatLib Library der Carnegie Mellon University erhältlich.  
Quelle: http://lib.stat.cmu.edu/datasets/houses.zip

Informieren Sie sich hier über den Inhalt des zu Grunde liegenden Datensatzes:\
https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset

Da der Datensatz sehr verbreitet ist, ist er im Machine Learning Paket _Scikit Learn_ enthalten.
Sie finden hier weitere Informationen zum Import des Datensatzes in Python:\
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html

Wir laden den California Housing Datensatz und laden ihn in einen Pandas Data Frame:

In [ ]:
from sklearn.datasets import fetch_california_housing
calhouse = fetch_california_housing(as_frame=True)
data = calhouse.frame
data.head()

In dieser Tabelle sehen wir die Features sowie das Zielattribut (_Target_) für eine Regressionsaufgabe. Das Zielattribut ist das Attribut, welches man später basierend auf weiteren Daten algorithmisch schätzen möchte. Die Werte des Zielattributs werden auch _Label_ genannt. In diesem Beispiel handelt es sich dabei um den Median-Hauswert (_MedHouseVal_) in einer californischen Census Blockgruppe. Jede Zeile repräsentiert also eine Census Blockgruppe. Eine Blockgruppe ist die kleinste geographische Einheit, für welche das US Census Bureau Daten veröffentlicht. Im Normalfall hat eine Blockgruppe eine Population von 600 bis 3.000 Menschen.

Die anderen Features sind die folgenden:

- **MedInc:**      Median-Einkommen
- **HouseAge:**    Median-Hausalter
- **AveRooms:**    Mittlere Zimmerzahl
- **AveBedrms:**   Mittlere Schlafzimmerzahl  
- **Population:**  Bevölkerungszahl
- **AveOccup:**    Mittlere Bewohnerzahl der Häuser
- **Latitude:**    Breitengrad
- **Longitude:**   Längengrad 

In einer Machine Learning Aufgabe möchte man nun an Hand dieser Daten lernen, wie der Zusammenhang zwischen diesen Features und dem zugehörigen Zielattribut (_MedHouseVal_: Median-Hauswert) ist.

Wir können uns wie folgt erst einmal ein paar Informationen über den Datensatz ausgeben lassen:

In [ ]:
data.info()

Wie wir sehen besteht der Datensatz aus 20640 Datenpunkten und 8 Features sowie dem Median-Hauswert (_MedHouseVal_) als vorherzusagendes Zielattribut.
Wir sehen außerdem, dass es in diesem Datensatz keine fehlenden Datenpunkte gibt. Der Datensatz ist zur Anwendung im Machine Learning also schon vorverarbeitet worden (so wie die meisten öffentlich zugänglichen Daten). Um den Realfall zu simulieren und unsere Data Engineering Methoden zu üben, müssen wir also unseren Datensatz künstlich verschlechtern.

In [ ]:
# Ändern einiger Datentypen
data.AveOccup = data.AveOccup.astype('string')
data.AveRooms = data.AveRooms.astype('string')
data.AveBedrms = data.AveBedrms.astype('int')

In [ ]:
# Bitte den Random Seed 0 nicht verändern, damit die Ergebnisse vergleichbar sind
rng = np.random.RandomState(0)

# Funktion zum Löschen einiger Datenpunkte
def add_missing_values(pdColumns,missing_rate):
    # Anzahl der Zeilen
    n_samples = pdColumns.shape[0]

    # Anzahl fehlender Werte basierend auf Zeilenanzahl und Fehlrate
    n_missing_samples = int(n_samples * missing_rate)

    # Erstellen eines bool arrays, welches an zufälligen Stellen 
    # n_missing_samples mal True enthält
    missing_samples = np.zeros(n_samples, dtype=bool)
    missing_samples[: n_missing_samples] = True
    rng.shuffle(missing_samples)
    
    # Erstellen der Ausgabevariable und löschen der entsprechenden Werte
    pdColumns_missing = pdColumns.copy()
    pdColumns_missing[missing_samples] = np.nan
    
    return pdColumns_missing

In [ ]:
# Füge bei einigen Features zufällig fehlende Werte hinzu
data.MedInc = add_missing_values(data.MedInc, 0.1)
data.Population = add_missing_values(data.Population, 0.01)
data.MedHouseVal = add_missing_values(data.MedHouseVal, 0.05)

# Füge bei einigen Features fehlende Werte am Stück hinzu
data.loc[354:635,('HouseAge')] = np.nan
data.loc[14657:14725,('HouseAge')] = np.nan

# Füge bei einigen Features Ausreißer hinzu
data.loc[[3,58,372,6927,10475],('Latitude')] = 0
data.loc[[368,3945,8375,14868,15968],('Longitude')] = 1000
data.loc[[23,465,746,1645],('HouseAge')] = 624
data.loc[[8,287,435,2865,5857,17586],('HouseAge')] = 814
data.loc[[385,938,6395,8156,15745, 17464],('HouseAge')] = 946

In [ ]:
data.info()

Wie wir sehen haben wir nun einen verschlechterten Datensatz vorliegen.

## Aufgabe (20 Punkte)

Machen Sie sich z.B. durch Visualisierungen/Plots mit den Daten vertraut und bereiten Sie die Tabelle für eine mögliche Datenanalyse vor.

* Welche Visualisierungen eignen sich?
* Was fällt Ihnen auf?
* Welche Probleme sehen Sie in diesem Datensatz?
* Welche Methoden können Sie verwenden, um den Datensatz für eine Datenanalyse vorzubereiten?

Entscheiden Sie sich für passende Datenvorverarbeitungsmethoden und bereiten Sie den Datensatz für eine Datenanalyse vor.
Erläutern Sie Ihr Vorgehen in Markdown Cells und/oder Python Kommentaren. Begründen Sie, welche Methoden Sie verwenden und notieren Sie Ihre Erkenntnisse.

## Bonusaufgabe (10 Extrapunkte)

Gehen Sie auf folgende Webseite:\
https://www.corona-daten-deutschland.de/

Gewinnen Sie dort unter **_Datensätze_** einen ersten Überblick über die verfügbaren Daten. 
Suchen Sie sich zwei Datensätze aus, welche Sie genauer untersuchen möchten. Notieren Sie in einer Markdown Cell im Notebook, welche Datensätze Sie verwenden.

* Importieren Sie beide Datensätze
* Visualisieren Sie beispielhaft einzelne Features
* _Joinen_ (verknüpfen) Sie beide Datensätze zu einer großen Tabelle (Pandas DataFrame), welche dann analysiert werden könnte
* Untersuchen Sie potentielle Korrelationen zwischen einzelnen Features, z.B. via Scatterplots oder Korrelationskoeffizienten

**Tipp**:\
csv's  können direkt in einen Pandas DataFrame geladen werden:\
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html